In [1]:
import datetime
import random
import shutil
import string
import time
from pathlib import Path
from pprint import pprint
from torchhd import HRRTensor
import joblib
import matplotlib.pyplot as plt
import pytorch_lightning as pl
import numpy as np
import pandas as pd
import torch
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import Callback, LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger
from sklearn.decomposition import PCA
from torch.utils.data import DataLoader, Dataset
from torch_geometric.data import Batch
from torch_geometric.datasets import ZINC

from graph_hdc.utils import AbstractEncoder
from src.datasets import AddNodeDegree
from src.encoding.configs_and_constants import HDCConfig, SupportedDataset
from src.encoding.graph_encoders import HyperNet
from src.encoding.the_types import VSAModel
from src.normalizing_flow.config import FlowConfig
from src.normalizing_flow.models import RealNVPLightning
from torch.utils.data import Subset

def setup_exp(ds_value: str) -> dict:
    """
    Sets up experiment directories based on the current script location.

    Args:
        ds_value (str): Dataset name to use for global_dataset_dir.

    Returns:
        dict: Dictionary containing paths to various directories.
    """
    # Resolve script location
    script_path = Path("/Users/arvandkaveh/Projects/kit/graph_hdc/notebooks/NormFlows/real_nvp.ipynb")
    experiments_path = script_path.parent
    script_stem = script_path.stem  # without .py

    # Resolve base and project directories
    base_dir = experiments_path / "results" / script_stem
    base_dir.mkdir(parents=True, exist_ok=True)

    project_dir = script_path.parents[2]  # adjust as needed

    print(f"Setting up experiment in {base_dir}")
    now = f"{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}_{''.join(random.choices(string.ascii_lowercase, k=4))}"
    exp_dir = base_dir / now
    exp_dir.mkdir(parents=True, exist_ok=True)
    print(f"Experiment directory created: {exp_dir}")

    dirs = {
        "exp_dir": exp_dir,
        "models_dir": exp_dir / "models",
        "evals_dir": exp_dir / "evaluations",
        "artefacts_dir": exp_dir / "artefacts",
        "global_model_dir": project_dir / "_models",
        "global_dataset_dir": project_dir / "_datasets" / ds_value,
    }

    for d in dirs.values():
        d.mkdir(parents=True, exist_ok=True)

    # Save a copy of the script
    try:
        shutil.copy(script_path, exp_dir / script_path.name)
        print(f"Saved a copy of the script to {exp_dir / script_path.name}")
    except Exception as e:
        print(f"Warning: Failed to save script copy: {e}")

    return dirs


def plot_train_val_loss(df, artefacts_dir):
    train = df[df["train_loss_epoch"].notna()]
    val = df[df["val_loss"].notna()]

    plt.figure(figsize=(8, 5))
    plt.plot(train["epoch"], train["train_loss_epoch"], label="Train Loss")
    plt.plot(val["epoch"], val["val_loss"], label="Validation Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Train vs. Validation Loss")
    plt.legend()
    plt.tight_layout()

    artefacts_dir.mkdir(exist_ok=True)
    plt.savefig(artefacts_dir / "train_val_loss.png")
    plt.close()
    print(f"Saved train/val loss plot to {artefacts_dir / 'train_val_loss.png'}")


def pca_encode(x: torch.Tensor, pca: PCA, norm: bool = False) -> torch.Tensor:
    """
    Encode data using a fitted PCA, with optional normalization.

    :param x: Input tensor of shape (..., features).
    :type x: torch.Tensor
    :param pca: Fitted PCA instance with attributes `mean_` and `std_`.
    :type pca: PCA
    :param norm: Whether to normalize input by mean and std before PCA.
    :type norm: bool
    :returns: Tensor of reduced dimensions, same dtype as input.
    :rtype: torch.Tensor

    The input is flattened over all but the last dimension, optionally normalized,
    transformed with the PCA, then returned as a tensor.
    """
    flat = x.view(-1, x.shape[-1]).cpu().numpy()
    if norm:
        flat = (flat - pca.mean_) / pca.std_
    reduced = pca.transform(flat)
    return torch.tensor(reduced, dtype=x.dtype)


def load_or_fit_pca(
        train_dataset: Dataset, encoder: AbstractEncoder, pca_path: Path | None = None, n_components: float = 0.99999,
        n_fit: int = 20000
) -> PCA:
    """
    Load an existing PCA from disk or fit a new one and save it.

    :param train_dataset: Dataset for fitting PCA.
    :type train_dataset: Dataset
    :param encoder: Model or function returning a dict with keys \"node_terms\", \"edge_terms\", and \"graph_embedding\".
    :param pca_path: Path to load/save the PCA object.
    :type pca_path: Path
    :param n_components: Number of components or variance ratio for PCA.
    :type n_components: float
    :param n_fit: Maximum number of samples to fit PCA on.
    :type n_fit: int
    :returns: Fitted PCA instance with `mean_` and `std_` attributes.
    :rtype: PCA

    If a PCA exists at `pca_path`, it is loaded. Otherwise, embeddings
    are collected by applying `encoder` to dataset entries until `n_fit`
    samples, flattened, and used to fit a new PCA. The mean and std of the
    fit data are stored on the PCA for later normalization.
    """
    if pca_path is not None and pca_path.exists():
        print(f"Loading existing PCA from {pca_path}")
        pca = joblib.load(pca_path)
        print(f"Loaded PCA with {pca.n_components_} components")
        return pca

    print("Fitting PCA on training data...")
    n_fit = min(n_fit, len(train_dataset))
    X_fit = []
    for i in range(n_fit):
        data = train_dataset[i]
        batch_data = Batch.from_data_list([data])
        res = encoder.forward(data=batch_data)
        x = torch.stack(
            [res["node_terms"].squeeze(0), res["edge_terms"].squeeze(0), res["graph_embedding"].squeeze(0)], dim=0
        )  # [3, D]
        X_fit.append(x.cpu().numpy())
    X_fit = np.stack(X_fit)
    X_fit_flat = X_fit.reshape(-1, X_fit.shape[-1])

    # Compute mean and std for normalization
    mu = np.mean(X_fit_flat, axis=0)
    sigma = np.std(X_fit_flat, axis=0)

    # Fit PCA
    pca = PCA(n_components=n_components, svd_solver="full")
    pca.fit(X_fit_flat)

    # Attach normalization stats
    pca.mean_ = mu
    pca.std_ = sigma

    print(f"PCA reduced dimension: {pca.n_components_} from {X_fit.shape[-1]}")
    joblib.dump(pca, pca_path)
    print(f"Saved new PCA to {pca_path}")
    return pca


def load_or_create_hypernet(path: Path, cfg: HDCConfig, depth: int) -> HyperNet:
    path = path / f"hypernet_{cfg.vsa.value}_d{cfg.hv_dim}_s{cfg.seed}_dpth{depth}.pt"
    if path.exists():
        print(f"Loading existing HyperNet from {path}")
        encoder = HyperNet(config=cfg, depth=depth)
        encoder.load(path)
    else:
        print("Creating new HyperNet instance.")
        encoder = HyperNet(config=cfg, depth=depth)
        encoder.populate_codebooks()
        encoder.save_to_path(path)
        print(f"Saved new HyperNet to {path}")
    return encoder


class EncodedPCADataset(torch.utils.data.Dataset):
    def __init__(self, base_dataset, encoder, pca: PCA | None = None, *, use_norm_pca: bool = False):
        self.base_dataset = base_dataset
        self.encoder = encoder
        self.pca = pca
        self.use_norm = use_norm_pca  # Whether to normalize the PCA

    def __len__(self):
        return len(self.base_dataset)

    def __getitem__(self, idx):
        data = self.base_dataset[idx]
        batch_data = Batch.from_data_list([data])
        res = self.encoder.forward(data=batch_data)
        x = torch.stack(
            [res["node_terms"].squeeze(0), res["edge_terms"].squeeze(0), res["graph_embedding"].squeeze(0)], dim=0
        )
        if self.pca is not None:
            return pca_encode(x, self.pca, self.use_norm)
        return x


def get_device():
    if torch.cuda.is_available():
        count = torch.cuda.device_count()
        print(f"CUDA is available. Detected {count} GPU device{'s' if count != 1 else ''}.")
        return torch.device("cuda")
    print("CUDA is not available.")
    return torch.device("cpu")


class TimeLoggingCallback(Callback):
    def setup(self, trainer, pl_module, stage=None):
        self.start_time = time.time()

    def on_train_epoch_end(self, trainer, pl_module):
        elapsed = time.time() - self.start_time
        trainer.logger.log_metrics({"elapsed_time_sec": elapsed}, step=trainer.current_epoch)


class DebugMetricsCallback(Callback):
    def on_train_epoch_end(self, trainer, pl_module):
        print("\n==> callback_metrics:")
        for k, v in trainer.callback_metrics.items():
            print(f"   {k:20s} → {type(v)}")
        print()


class SamplingEveryNEpoch(Callback):
    def __init__(self, encoder, n_samples: int = 10, every_n_epochs: int = 10):
        """
        :param encoder: your graph‐reconstruction encoder, with `.decode_order_zero_counter()`
        :param n_samples: how many samples to draw
        :param every_n_epochs: interval at which to run the sampling
        """
        super().__init__()
        self.encoder = encoder
        self.n_samples = n_samples
        self.every_n = every_n_epochs

    def on_train_epoch_end(self, trainer, pl_module):
        epoch = trainer.current_epoch
        if epoch % self.every_n != 0:
            return

        device = pl_module.device
        self.encoder.to(device)

        model.eval()
        with torch.no_grad():
            print(f"\n=== Sampling callback at epoch {epoch} ===")
            # ---- Sampling example ----
            z, logs = pl_module.sample(self.n_samples)
            z = z.to(device)

            # if you had a PCA decode step:
            # z = pca_decode(z, pca)

            # unpack into (batch, 3, D)
            node_terms_s, _, _ = z.unbind(dim=1)

            # Cast to HRRTensor
            node_terms_hrr = node_terms_s.as_subclass(HRRTensor)

            for b in range(self.n_samples):
                print(f"-- SAMPLE #{b} --")
                node_counter = self.encoder.decode_order_zero_counter(node_terms_hrr[b])
                print(f"node_counter[0] = {node_counter[0]}")
                print(f"  total = {node_counter[0].total()}")
        pl_module.train()


hv_dim = 6400
batch_size = 4
cfg = FlowConfig(
    project_dir="/Users/arvandkaveh/Projects/kit/graph_hdc",
    seed=42,
    epochs=20,
    batch_size=16,
    vsa=VSAModel.HRR,
    hv_dim=hv_dim,
    dataset=SupportedDataset.ZINC_NODE_DEGREE_COMB,
    num_input_channels=3 * hv_dim,
    num_flows=12,
    num_hidden_channels=128,
    input_shape=(3, hv_dim),
    device="cpu",
    lr=0.00003,
    weight_decay=0.0001,
)

print("Running experiment")
pprint(cfg.__dict__, indent=2)

dirs = setup_exp(cfg.dataset.value)
exp_dir = dirs["exp_dir"]
models_dir = dirs["models_dir"]
evals_dir = dirs["evals_dir"]
artefacts_dir = dirs["artefacts_dir"]
global_model_dir = dirs["global_model_dir"]
global_dataset_dir = dirs["global_dataset_dir"]

# W&B Logging — use existing run (from sweep or manual init)
# run = wandb.run or wandb.init(project="realnvp-hdc", config=cfg.__dict__, name=f"run_{cfg.hv_dim}_{cfg.seed}", reinit=True)
# run.tags = [f"hv_dim={cfg.hv_dim}", f"vsa={cfg.vsa.value}", f"dataset={cfg.dataset.value}"]

# wandb_logger = WandbLogger(log_model=True, experiment=run)

train_dataset = ZINC(root=str(global_dataset_dir), pre_transform=AddNodeDegree(), split="train", subset=True)[:1]
# make a length-4 dataset by selecting index 0 four times
train_dataset = Subset(train_dataset, indices=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
print(f"{len(train_dataset)=}")
# validation_dataset = ZINC(root=str(global_dataset_dir), pre_transform=AddNodeDegree(), split="val", subset=True)[:1]
# print(f"{len(validation_dataset)=}")

device = get_device()
ds = cfg.dataset
ds.default_cfg.vsa = cfg.vsa
ds.default_cfg.hv_dim = cfg.hv_dim
ds.default_cfg.device = device
ds.default_cfg.seed = cfg.seed
ds.default_cfg.edge_feature_configs = {}
ds.default_cfg.graph_feature_configs = {}

encoder = load_or_create_hypernet(path=global_model_dir, cfg=ds.default_cfg, depth=3)

n_components = 0.99
pca_path = global_model_dir / f"hypervec_pca_{cfg.vsa.value}_d{cfg.hv_dim}_s{cfg.seed}_c{str(n_components)[2:]}.joblib"
pca = load_or_fit_pca(
    train_dataset=ZINC(root=str(global_dataset_dir), pre_transform=AddNodeDegree(), split="train", subset=True),
    encoder=encoder,
    pca_path=pca_path,
    n_components=n_components,
    n_fit=20_000,
)

# reduced_dim = int(pca.n_components_)
cfg.num_input_channels = 3 * hv_dim
cfg.input_shape = (3, hv_dim)

train_dataloader = DataLoader(
    EncodedPCADataset(train_dataset, encoder),
    batch_size=cfg.batch_size,
    shuffle=True,
    num_workers=0,
    pin_memory=torch.cuda.is_available(),
    drop_last=True,
)
#
# validation_dataloader = DataLoader(
#     EncodedPCADataset(validation_dataset, encoder),
#     batch_size=cfg.batch_size,
#     shuffle=False,
#     num_workers=0,
#     pin_memory=torch.cuda.is_available(),
#     drop_last=False,
# )

model = RealNVPLightning(cfg)

csv_logger = CSVLogger(save_dir=str(evals_dir), name="logs")
checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    save_top_k=2,
    mode="min",
    dirpath=str(models_dir),
    filename="epoch{epoch:02d}-val{val_loss:.2f}",
    save_last=True,
)
lr_monitor = LearningRateMonitor(logging_interval="epoch")
time_logger = TimeLoggingCallback()
# debug_callback = DebugMetricsCallback()
sampling_cb = SamplingEveryNEpoch(encoder=encoder, n_samples=10, every_n_epochs=10)

trainer = Trainer(
    max_epochs=cfg.epochs,
    logger=[csv_logger,
            # wandb_logger
            ],
    callbacks=[checkpoint_callback, lr_monitor, time_logger,
               # debug_callback,

               ],
    default_root_dir=str(exp_dir),
    accelerator="auto",
    log_every_n_steps=20,
    enable_progress_bar=True,
    detect_anomaly=True,
)

trainer.fit(model,
            train_dataloaders=train_dataloader,
            # val_dataloaders=validation_dataloader
            )

torch.save(model.state_dict(), models_dir / "final_model.pt")

metrics_path = Path(csv_logger.log_dir) / "metrics.csv"
if metrics_path.exists():
    df = pd.read_csv(metrics_path)
    df.to_parquet(evals_dir / "metrics.parquet")
    plot_train_val_loss(df, artefacts_dir)

print("==== The Experiment is done! ====")


Running experiment
{ 'activation': <class 'torch.nn.modules.activation.ReLU'>,
  'batch_size': 16,
  'dataset': <SupportedDataset.ZINC_NODE_DEGREE_COMB: 'ZINC_ND_COMB'>,
  'device': 'cpu',
  'dropout_probability': 0.0,
  'epochs': 20,
  'exp_dir': None,
  'flow_type': <class 'normflows.flows.neural_spline.wrapper.AutoregressiveRationalQuadraticSpline'>,
  'hv_dim': 6400,
  'init_identity': True,
  'input_shape': (3, 6400),
  'lr': 3e-05,
  'num_bins': 8,
  'num_blocks': 2,
  'num_context_channels': None,
  'num_flows': 12,
  'num_hidden_channels': 128,
  'num_input_channels': 19200,
  'permute': False,
  'project_dir': PosixPath('/Users/arvandkaveh/Projects/kit/graph_hdc'),
  'seed': 42,
  'tail_bound': 3,
  'vsa': <VSAModel.HRR: 'HRR'>,
  'weight_decay': 0.0001}
Setting up experiment in /Users/arvandkaveh/Projects/kit/graph_hdc/notebooks/NormFlows/results/real_nvp
Experiment directory created: /Users/arvandkaveh/Projects/kit/graph_hdc/notebooks/NormFlows/results/real_nvp/2025-07-14_15

You have turned on `Trainer(detect_anomaly=True)`. This will significantly slow down compute speed and is recommended only for model debugging.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/arvandkaveh/Projects/kit/graph_hdc/.pixi/envs/default/lib/python3.13/site-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.

  | Name | Type            | Params | Mode 
-------------------------------------------------
0 | flow | NormalizingFlow | 118 M  | train
-------------------------------------------------
118 M     Trainable params
0         Non-trainable params
118 M     Total params
475.300   Total estimated model params size (MB)
195       Modules in train mode
0         Modules in eval mode
/Users/arvandkaveh/Projects/kit/graph_hdc/.pixi/envs/default/lib/python3.13/site-packages/pytorch_lightning/trainer/connectors/data

Epoch 0: 100%|██████████| 1/1 [00:01<00:00,  0.60it/s, v_num=0, train_loss_step=3.04e+9]
=== Sampling callback at epoch 0 ===
-- SAMPLE #0 --
node_counter[0] = Counter({(19, 0): 3, (0, 2): 2, (4, 1): 2, (5, 2): 2, (20, 3): 2, (21, 1): 2, (21, 2): 2, (24, 4): 2, (2, 0): 1, (2, 2): 1, (4, 4): 1, (4, 5): 1, (5, 3): 1, (5, 4): 1, (5, 5): 1, (6, 1): 1, (6, 3): 1, (7, 0): 1, (7, 4): 1, (7, 5): 1, (9, 1): 1, (9, 2): 1, (9, 3): 1, (10, 0): 1, (11, 0): 1, (11, 5): 1, (12, 4): 1, (14, 0): 1, (14, 2): 1, (14, 4): 1, (17, 0): 1, (17, 3): 1, (18, 4): 1, (18, 5): 1, (20, 0): 1, (20, 1): 1, (20, 4): 1, (21, 0): 1, (22, 2): 1, (23, 0): 1, (23, 1): 1, (23, 3): 1, (23, 4): 1, (24, 0): 1, (24, 5): 1, (25, 1): 1, (25, 4): 1, (26, 1): 1, (26, 3): 1, (27, 4): 1, (27, 5): 1})
  total = 60
-- SAMPLE #1 --
node_counter[0] = Counter({(18, 2): 3, (0, 2): 2, (0, 4): 2, (4, 3): 2, (4, 4): 2, (5, 0): 2, (5, 3): 2, (12, 3): 2, (12, 4): 2, (16, 0): 2, (18, 4): 2, (22, 2): 2, (26, 2): 2, (1, 1): 1, (2, 2): 1, (2, 5): 

/Users/arvandkaveh/Projects/kit/graph_hdc/.pixi/envs/default/lib/python3.13/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:384: `ModelCheckpoint(monitor='val_loss')` could not find the monitored key in the returned metrics: ['lr-Adam', 'train_loss', 'train_loss_step', 'lr', 'train_loss_epoch', 'epoch', 'step']. HINT: Did you call `log('val_loss', value)` in the `LightningModule`?


Epoch 10: 100%|██████████| 1/1 [00:00<00:00,  1.26it/s, v_num=0, train_loss_step=1.52e+9, lr=3e-5, train_loss_epoch=1.52e+9]
=== Sampling callback at epoch 10 ===
-- SAMPLE #0 --
node_counter[0] = Counter({(0, 0): 3, (6, 0): 3, (18, 5): 3, (1, 4): 2, (2, 0): 2, (2, 4): 2, (3, 5): 2, (5, 1): 2, (7, 2): 2, (8, 0): 2, (10, 4): 2, (14, 1): 2, (17, 1): 2, (22, 3): 2, (24, 2): 2, (25, 5): 2, (0, 5): 1, (2, 2): 1, (3, 4): 1, (4, 0): 1, (5, 4): 1, (6, 4): 1, (6, 5): 1, (7, 3): 1, (7, 5): 1, (8, 4): 1, (9, 4): 1, (10, 2): 1, (10, 5): 1, (12, 2): 1, (13, 1): 1, (14, 2): 1, (15, 0): 1, (15, 4): 1, (15, 5): 1, (17, 4): 1, (18, 1): 1, (18, 3): 1, (19, 3): 1, (19, 5): 1, (20, 0): 1, (20, 2): 1, (21, 0): 1, (21, 2): 1, (23, 1): 1, (23, 3): 1, (23, 4): 1, (24, 3): 1, (24, 4): 1, (24, 5): 1, (25, 3): 1, (26, 2): 1, (27, 1): 1, (27, 2): 1})
  total = 73
-- SAMPLE #1 --
node_counter[0] = Counter({(8, 4): 3, (15, 5): 3, (0, 0): 2, (10, 1): 2, (27, 4): 2, (1, 0): 1, (2, 1): 1, (2, 3): 1, (2, 5): 1, (3, 2):

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 1/1 [00:02<00:00,  0.40it/s, v_num=0, train_loss_step=1.52e+9, lr=3e-5, train_loss_epoch=1.52e+9]


KeyError: 'val_loss'

In [2]:
def pca_decode(x_reduced: torch.Tensor, pca: PCA, denorm: bool = False) -> torch.Tensor:
    """
    Decode PCA-reduced data, with optional de-normalization.

    :param x_reduced: Reduced tensor of shape (..., reduced_features).
    :type x_reduced: torch.Tensor
    :param pca: Fitted PCA instance with attributes `mean_` and `std_`.
    :type pca: PCA
    :param denorm: Whether to reverse normalization after inverse PCA.
    :type denorm: bool
    :returns: Reconstructed tensor in original feature space.
    :rtype: torch.Tensor

    The reduced tensor is flattened, inverse-transformed by PCA, and then
    optionally scaled and shifted back.
    """
    flat = x_reduced.view(-1, x_reduced.shape[-1]).detach().cpu().numpy()
    recon = pca.inverse_transform(flat)
    if denorm:
        recon = recon * pca.std_ + pca.mean_
    return torch.tensor(recon, dtype=x_reduced.dtype)


In [2]:

model_loaded = RealNVPLightning.load_from_checkpoint("/Users/arvandkaveh/Projects/kit/graph_hdc/notebooks/NormFlows/results/real_nvp/2025-07-14_15-27-09_gieh/models/last.ckpt")

# ---- Sampling example ----
n_samples = 10
model_loaded.eval()
s, l = model_loaded.sample(n_samples)
encoder.to(model_loaded.device)
# s_decoded = pca_decode(s, pca)
# z = s_decoded.view(n_samples, 3, hv_dim)
z = s

In [3]:

import torch
from torchhd import HRRTensor

node_terms_s, edge_terms_s, graph_embeddings_s = z.unbind(dim=1)

# Cast to HRRTensor
node_terms_s_hrr = node_terms_s.as_subclass(HRRTensor)
edge_terms_s_hrr = edge_terms_s.as_subclass(HRRTensor)
graph_embeddings_s_hrr = graph_embeddings_s.as_subclass(HRRTensor)

print("---SAMPLES---")
for b in range(n_samples):
    print(f"SAMPLE NR: {b}")
    node_counter_dec = encoder.decode_order_zero_counter(node_terms_s_hrr[b])
    print(f"{node_counter_dec[0]=}")
    print(f"{node_counter_dec[0].total()}")

import torchhd

print("----RANDOM-----")
random_counter = encoder.decode_order_zero_counter(torchhd.random(1, 6400, vsa="HRR")[0])
print(f"{random_counter[0]=}")
print(f"{random_counter[0].total()=}")


---SAMPLES---
SAMPLE NR: 0
node_counter_dec[0]=Counter({(1, 3): 2, (4, 2): 2, (6, 4): 2, (9, 5): 2, (14, 1): 2, (17, 0): 2, (17, 4): 2, (19, 1): 2, (24, 5): 2, (25, 2): 2, (0, 0): 1, (0, 2): 1, (0, 4): 1, (0, 5): 1, (2, 2): 1, (5, 0): 1, (6, 0): 1, (6, 1): 1, (7, 2): 1, (7, 3): 1, (8, 1): 1, (9, 0): 1, (9, 4): 1, (10, 2): 1, (11, 0): 1, (11, 3): 1, (12, 0): 1, (12, 3): 1, (13, 1): 1, (14, 5): 1, (15, 3): 1, (16, 5): 1, (17, 2): 1, (19, 0): 1, (20, 2): 1, (21, 0): 1, (22, 1): 1, (22, 3): 1, (22, 5): 1, (23, 0): 1, (24, 3): 1, (25, 0): 1, (25, 4): 1, (26, 5): 1, (27, 2): 1, (27, 5): 1})
56
SAMPLE NR: 1
node_counter_dec[0]=Counter({(7, 2): 3, (12, 5): 3, (23, 4): 3, (1, 0): 2, (2, 2): 2, (3, 4): 2, (4, 0): 2, (4, 1): 2, (8, 2): 2, (11, 3): 2, (12, 3): 2, (13, 4): 2, (13, 5): 2, (14, 1): 2, (17, 4): 2, (25, 1): 2, (25, 5): 2, (26, 5): 2, (0, 0): 1, (0, 1): 1, (1, 2): 1, (2, 4): 1, (2, 5): 1, (3, 2): 1, (4, 4): 1, (4, 5): 1, (5, 0): 1, (6, 3): 1, (7, 3): 1, (7, 5): 1, (8, 0): 1, (8, 1): 1, 

RuntimeError: Tensor for argument #1 'mat1' is on CPU, but expected it to be on GPU (while checking arguments for mm)

In [7]:

print("---DATA---")
counter = 1
for b in train_dataloader:
    print(f"DATA NR: {counter}")
    counter += 1
    # s_decoded = pca_decode(b, pca)
    z = b.view(16, 3, hv_dim)
    node_terms_s, edge_terms_s, graph_embeddings_s = z.unbind(dim=1)
    node_terms_s_hrr = node_terms_s.as_subclass(HRRTensor)
    for i in range(4):
        encoder.to(node_terms_s_hrr.device)
        node_counter_dec = encoder.decode_order_zero_counter(node_terms_s_hrr[i])
        print(f"{node_counter_dec[0]=}")
        print(f"{node_counter_dec[0].total()}")


---DATA---
DATA NR: 1
node_counter_dec[0]=Counter({(0, 2): 13, (0, 3): 6, (1, 1): 3, (2, 3): 3, (0, 1): 2, (1, 2): 1, (5, 4): 1})
29
node_counter_dec[0]=Counter({(0, 2): 13, (0, 3): 6, (1, 1): 3, (2, 3): 3, (0, 1): 2, (1, 2): 1, (5, 4): 1})
29
node_counter_dec[0]=Counter({(0, 2): 13, (0, 3): 6, (1, 1): 3, (2, 3): 3, (0, 1): 2, (1, 2): 1, (5, 4): 1})
29
node_counter_dec[0]=Counter({(0, 2): 13, (0, 3): 6, (1, 1): 3, (2, 3): 3, (0, 1): 2, (1, 2): 1, (5, 4): 1})
29


In [ ]:
model_loaded = RealNVPLightning.load_from_checkpoint(
    "/Users/arvandkaveh/Projects/kit/graph_hdc/notebooks/NormFlows/results/real_nvp/2025-07-12_10-33-51_wews/models/epochepoch=229-valval_loss=-14874.93.ckpt")
